In [1]:
import numpy as np
import copy
import itertools

from qec_generator import CircuitParams
from simulate_qec_rounds_stim import experiment_run, monte_carlo_experiment_run
from stim_lib.scheduled_circuit import generate_scheduled, get_pauli_probs

from quantumsim import circuit, ptm, sparsedm

In [2]:
cparams = CircuitParams(t1=15e3,
                        t2=19e3,
                        single_qubit_gate_duration=14,
                        two_qubit_gate_duration=26,
                        single_qubit_depolarization_rate=0.0,#1.1e-3,
                        two_qubit_depolarization_rate=0.0,#6.6e-3,
                        meas_duration=600,
                        reset_duration=0,
                        reset_latency=40,
                        meas_induced_dephasing_enhancement=3)

In [3]:
task = 'surface_code:rotated_memory_z'  # looks ok
distance = 2
rounds = 1
reset_strategy = 'AR'

circ, cont, _ = generate_scheduled(
    code_task='surface_code:rotated_memory_z',  # looks ok
    distance=distance,
    rounds=rounds,
    params=cparams,
    disable_ancilla_reset=False,
    separate_gate_errors=False,
    meas_induced_dephasing_enhancement=False)
res1 = experiment_run(circ, cont, shots=2000, reset_strategy=reset_strategy)

print(res1)

0.9545


In [4]:
res2 = monte_carlo_experiment_run(task, distance, rounds, cparams,
                                  shots=2000, reset_strategy=reset_strategy)
print(res2)

0.45872127884981284


## Implementation in quantumsim

In [5]:
circ

stim.Circuit('''
    QUBIT_COORDS(1, 1) 1
    QUBIT_COORDS(2, 0) 2
    QUBIT_COORDS(3, 1) 3
    QUBIT_COORDS(1, 3) 6
    QUBIT_COORDS(2, 2) 7
    QUBIT_COORDS(3, 3) 8
    QUBIT_COORDS(2, 4) 12
    R 1 3 6 8 2 7 12
    PAULI_CHANNEL_1(0.000665779, 0.000665779, 0.000385746)
    TICK
    H 2 12
    PAULI_CHANNEL_1(0.000233224, 0.000233224, 0.000135061) 1 3 6 7 8
    TICK
    CX 2 3 8 7
    PAULI_CHANNEL_1(0.000432958, 0.000432958, 0.000250785) 1 6 12
    TICK
    CX 2 1 3 7
    PAULI_CHANNEL_1(0.000432958, 0.000432958, 0.000250785) 6 8 12
    TICK
    CX 12 8 6 7
    PAULI_CHANNEL_1(0.000432958, 0.000432958, 0.000250785) 1 2 3
    TICK
    CX 12 6 1 7
    PAULI_CHANNEL_1(0.000432958, 0.000432958, 0.000250785) 2 3 8
    TICK
    H 2 12
    PAULI_CHANNEL_1(0.000233224, 0.000233224, 0.000135061) 1 3 6 7 8
    TICK
    MR 2 7 12
    PAULI_CHANNEL_1(0.0104423, 0.0104423, 0.00611929) 1 3 6 8
    DETECTOR(2, 2, 0) rec[-2]
    M 1 3 6 8
    PAULI_CHANNEL_1(0.00980264, 0.00980264, 0.00574013) 2 7 

 ## detector indices
 
 1   2   3   6   7   8   12
 
 
-7  -6  -5  -4  -3  -2   -1

In [6]:
def get_probs(duration, t1, t2):
    gamma = 1 - np.exp(-duration/t1)
    return gamma, (1 - gamma)*(1 - np.exp(-2*duration/t2))  

def get_kraus_ops(duration, t1, t2):
    ## see Eqn 11 in https://arxiv.org/pdf/1305.2021.pdf
    gamma, lbda = get_probs(duration, t1, t2)
    E1 = np.array([[1, 0],[0, np.sqrt(1 - gamma - lbda)]])
    E2 = np.array([[0, np.sqrt(gamma)],[0, 0]])
    E3 = np.array([[0, 0],[0, np.sqrt(lbda)]])
    return E1, E2, E3

In [7]:
def t1t2_ptm(duration, t1, t2):
    ops = get_kraus_ops(duration, t1, t2)
    _ptm = 0
    for op in ops:
        _ptm += ptm.single_kraus_to_ptm(op)
    return _ptm

In [8]:
qbit_ids = [1, 2, 3, 6, 7, 8, 12]
qbits = [str(i) for i in qbit_ids]
cbits = ["m"+str(i) for i in qbit_ids]

t_single = cparams.single_qubit_gate_duration
t_double = cparams.two_qubit_gate_duration
t_meas = cparams.meas_duration
t_meas_reset = cparams.meas_duration + cparams.reset_duration +  cparams.reset_latency

reset_noise = t1t2_ptm(cparams.reset_duration + cparams.reset_latency,
                                    cparams.t1, cparams.t2)
idle_noise_1qb_gate = t1t2_ptm(t_single, cparams.t1, cparams.t2)
idle_noise_2qb_gate = t1t2_ptm(t_double, cparams.t1, cparams.t2)
meas_noise = t1t2_ptm(t_meas, cparams.t1, cparams.t2)
meas_reset_noise = t1t2_ptm(t_meas_reset, cparams.t1, cparams.t2)

In [19]:
t_start = 0

sampler = circuit.BiasedSampler(0, 1, seed=42)

c = circuit.Circuit()
state = sparsedm.SparseDM(qbits+cbits)

for i in qbits:
    c.add_qubit(circuit.Qubit(i))
for i in cbits:
    c.add_qubit(circuit.ClassicalBit(i))
    
for qb in qbits:
    c.add_gate(circuit.SinglePTMGate(qb, t_start, reset_noise))

for qb in ["2", "12"]:
    c.add_gate(circuit.Hadamard(qb, t_start + t_single/2))
for qb in ["1", "3", "6", "7", "8"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+ t_single/2,
                                     idle_noise_1qb_gate))  
t_start += t_single

for qbs in [("2", "3"), ("8", "7")]:
    c.add_gate(circuit.CNOT(qbs[0], qbs[1], t_start+t_double/2))
for qb in ["1", "6", "12"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+t_double/2,
                                     idle_noise_2qb_gate
                                    ))
t_start += t_double

for qbs in [("2", "1"), ("3", "7")]:
    c.add_gate(circuit.CNOT(qbs[0], qbs[1], t_start+t_double/2))
for qb in ["6", "8", "12"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+t_double/2,
                                     idle_noise_2qb_gate
                                    ))
t_start += t_double


for qbs in [("12", "8"), ("6", "7")]:
    c.add_gate(circuit.CNOT(qbs[0], qbs[1], t_start+t_double/2))    
for qb in ["1", "2", "3"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+t_double/2,
                                     idle_noise_2qb_gate
                                    ))
t_start += t_double

for qbs in [("12", "6"), ("1", "7")]:
    c.add_gate(circuit.CNOT(qbs[0], qbs[1], t_start+t_double/2))
for qb in ["2", "3", "8"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+t_double/2,
                                     idle_noise_2qb_gate
                                    ))
t_start += t_double

for qb in ["2", "12"]:
    c.add_gate(circuit.Hadamard(qb, t_start + t_single/2))
for qb in ["1", "3", "6", "7", "8"]:
    c.add_gate(circuit.SinglePTMGate(qb, t_start+ t_single/2,
                                     idle_noise_1qb_gate
                                     ))  
t_start += t_single

for qb in ["2", "7", "12"]:
    c.add_gate(circuit.Measurement(qb, 
                                   t_start + t_meas_reset/2,
                                   sampler = sampler,
                                   output_bit = 'm'+qb))
for qb in ["1", "3", "6", "8"]:
    c.add_gate(circuit.SinglePTMGate(qb,
                                     t_start + t_meas_reset/2,
                                     meas_reset_noise))    
    
c.order()
c.apply_to(state)
print(state.classical)

c.gates = []#empty the gates

for qb in ["2", "7", "12"]:
    if state.classical['m'+qb]:
        c.add_gate(circuit.X(qb, t_start + t_meas_reset/2 + 0.1))
                      
                                   
t_start += t_meas_reset


for qb in ["1", "3", "6", "8"]:
    c.add_gate(circuit.Measurement(qb, 
                                   t_start + t_meas/2,
                                   sampler = sampler,
                                   output_bit = 'm'+qb))
for qb in ["2", "7", "12"]:
    c.add_gate(circuit.SinglePTMGate(qb,
                                    t_start + t_meas/2,
                                    meas_noise))
t_start += t_meas

c.order() 
c.apply_to(state)

print(state.classical)

{'m1': 0, 'm2': 0, 'm3': 0, 'm6': 0, 'm7': 0, 'm8': 0, 'm12': 0, '2': 0, '7': 0, '12': 0}
{'m1': 0, 'm2': 0, 'm3': 0, 'm6': 0, 'm7': 0, 'm8': 0, 'm12': 0, '8': 0, '6': 0, '3': 0, '1': 0}
